# Character-based RNN

**Prepare the packages and import the data**

In [0]:
import re
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import os
import string
import nltk
f = open("../data/processed/popSongs.txt", encoding="utf-8")
text = f.read(1500000)
#text = re.sub(r'[^\x00-\x7f]',r'', text) 
text = re.sub(r'[^a-zA-Z\s\x27]', '', text)

**Preprocess the data**

In [0]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
charToIndex = {u:i for i, u in enumerate(vocab)}
indexToChar = np.array(vocab)
textAsIndices = np.array([charToIndex[c] for c in text])

In [0]:
songLen = 100

# Create training examples / targets
ds = tf.data.Dataset.from_tensor_slices(textAsIndices)

# Convert characters to sequences
sequences = ds.batch(songLen+1, drop_remainder=True)

In [0]:
def getTrainAndTarget(seq):
    train = seq[:-1]
    target = seq[1:]
    return train, target

trainData = sequences.map(getTrainAndTarget)

**Make the model**

In [0]:
""" Create training batches"""
batchSize = 64
bufferSize = 10000
trainData = trainData.shuffle(bufferSize).batch(batchSize, drop_remainder=True)

""" Build the model """
vocabSize = len(vocab)
embeddingDim = 256
units = 1024

In [0]:
def buildModel(units, vocabSize, embeddingDim, batchSize):
  model = Sequential()
  model.add(Embedding(vocabSize, embeddingDim, batch_input_shape=[batchSize, None]))
  model.add(GRU(units, return_sequences=True, stateful=True,recurrent_initializer='glorot_uniform'))

  #model.add(LSTM(units, return_sequences=True))
  #model.add(Dropout(0.8))

  model.add(Dense(vocabSize))
  
  return model

model = buildModel(units=units, vocabSize=vocabSize, embeddingDim=embeddingDim, batchSize=batchSize)

**Train the model**

In [0]:
def loss(trueData, predictions):
    return tf.keras.losses.sparse_categorical_crossentropy(trueData, predictions, from_logits=True)

#Note: this is computed using an exponential base
def perplexity(trueData, prediction):
  cross_entropy = tf.keras.backend.mean(tf.keras.backend.sparse_categorical_crossentropy(trueData, prediction))
  perplexity = tf.keras.backend.exp(cross_entropy)
  return perplexity

In [0]:
model.compile(optimizer='adam', loss=loss, metrics=[perplexity])

#Save the checkpoints in a new file during the training
checkpointDirectory = './training_checkpoints'
checkpointFilePath = os.path.join(checkpointDirectory, "ckpt_{epoch}")
checkpointCallback = ModelCheckpoint(filepath=checkpointFilePath, save_weights_only=True)

In [0]:
model.fit(trainData, epochs=50, callbacks=[checkpointCallback])

Epoch 1/50
223/223 [==============================] - 13s 56ms/step - loss: 2.5197 - perplexity: 77.0383
Epoch 2/50
223/223 [==============================] - 13s 58ms/step - loss: 1.8483 - perplexity: 21.3353
Epoch 3/50
223/223 [==============================] - 13s 57ms/step - loss: 1.5823 - perplexity: 18.8426
Epoch 4/50
223/223 [==============================] - 13s 57ms/step - loss: 1.4283 - perplexity: 17.5782
Epoch 5/50
223/223 [==============================] - 12s 56ms/step - loss: 1.3196 - perplexity: 16.7352
Epoch 6/50
223/223 [==============================] - 12s 56ms/step - loss: 1.2283 - perplexity: 16.0945
Epoch 7/50
223/223 [==============================] - 13s 56ms/step - loss: 1.1459 - perplexity: 15.5633
Epoch 8/50
223/223 [==============================] - 13s 56ms/step - loss: 1.0714 - perplexity: 15.1939
Epoch 9/50
223/223 [==============================] - 13s 56ms/step - loss: 1.0007 - perplexity: 14.8972
Epoch 10/50
223/223 [==============================] - 

In [0]:
#To rebuild model and restore the weights from the previous checkpoint
tf.train.latest_checkpoint(checkpointDirectory)
model = buildModel(units, vocabSize, embeddingDim, batchSize=1)
model.load_weights(tf.train.latest_checkpoint(checkpointDirectory))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (1, None, 256)            14592     
_________________________________________________________________
gru_15 (GRU)                 (1, None, 1024)           3938304   
_________________________________________________________________
dense_21 (Dense)             (1, None, 57)             58425     
Total params: 4,011,321
Trainable params: 4,011,321
Non-trainable params: 0
_________________________________________________________________



**Generate the song**

In [0]:
def generateSong(model, inputString, songLen):

  #Convert input to indices
  inputToIndex = [charToIndex[s] for s in inputString]
  inputToIndex = tf.expand_dims(inputToIndex, 0)

  song = []

  model.reset_states()
  for i in range(songLen):
      pred = model(inputToIndex)
      pred = tf.squeeze(pred, 0)

      #Temperature sampling
      pred = temperatureSampling(0.5, pred)

      #Nucleus sampling
      #pred = nucleusSampling(0.95, pred)

      #Greedy
      #predIndex = tf.argmax(pred)


      predIndex = tf.random.categorical(pred, num_samples=1)[-1,0].numpy()

      # Set input to predicted character and previous hidden state
      inputToIndex = tf.expand_dims([predIndex], 0)

      song.append(indexToChar[predIndex])

  song = inputString + ''.join(song)
  return song

In [0]:
def temperatureSampling(temperature, predictions):
  return predictions/temperature


In [0]:
def nucleusSampling(p, predictions):
    sortedPreds = tf.sort(predictions, direction='DESCENDING')
    sortedProbs = tf.nn.softmax(sortedPreds)
    sumOfProbs = tf.cumsum(sortedProbs, axis=1, exclusive=True)
    #Cutoff the preds that have a CDF < p
    predsToRemove = tf.where(sumOfProbs < p, sortedPreds, 
                                 tf.ones_like(sortedPreds)*1000)
    minPreds = tf.reduce_min(predsToRemove, axis=1, keepdims=True) 
    return tf.where(predictions < minPreds,
            tf.ones_like(predictions, dtype=predictions.dtype) * -1e10, predictions)

In [0]:
generatedSong = generateSong(model, inputString=" ", songLen=1000)
print(generatedSong)

 I have loved you and not feel your hand
Just don't stand that I breathe one thing I need
I don't wanna get you on
I won't take my medicine
I took a home on powers
'cause I'm the camera now
Oh oh oh oh oh oh oh oh oh oh
You should leave with me
I want to know that I love you like that
I would make you my will open things you know you ain't got more than just a little bit
If you want my body gettin' bodied
Gettin' bodied gettin' bodied
You want my body
Tonight I'll be your naughty girl I'm
Callin' all my girls
I see you look me up and down
And I came to party
Tonight I'll be your naughty girl
I'm callin' all my girls
I see you look me up and down
And I came to party I'm in our home
And now the light back home
You can find it happy just to call you a can't live
without him up wet
Baby who can something sleep no distante poe un tormirte de mi
Yo tengo que encontre besaban
All the world will care
All the wailight on the floor
You know I'm the type of girl
World
Interno in love with you
Oh 


**Evaluate the model**

In [0]:
def selfBleuScore(generatedSong, seq_length):
  generatedNgrams = []
  generatedSong = generatedSong.split()
  for i in range(seq_length,len(generatedSong)):
      generatedNgrams.append(generatedSong[i-seq_length:i])
  f = nltk.translate.bleu_score.SmoothingFunction()
  bleuScores = []
  for genNgram in generatedNgrams:
    for genNgram2 in generatedNgrams:
      if genNgram != genNgram2:
        bleuScores.append(nltk.translate.bleu_score.sentence_bleu([genNgram], genNgram2, smoothing_function=f.method1))
  return sum(bleuScores)/len(bleuScores)

print(selfBleuScore(generatedSong, 5))

0.023252292216035263


Compute reference Self-BLEU Score of songs in dataset

In [0]:
import re
import tensorflow as tf
import numpy as np
import os
import string
import nltk
f = open("../data/processed/popSongs2.txt", encoding="utf-8")
text = f.read(1500000)
text = re.sub(r'[^a-zA-Z\s\x27]', '', text)
songs = text.split('\n')

In [0]:
selfBleuScores = []
for i in range(100):
  selfBleuScores.append(selfBleuScore(songs[i], 5))
referenceBleuScore = sum(selfBleuScores)/len(selfBleuScores)
print(referenceBleuScore)

0.03217095640055971
